In [166]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flight-delays-spring-2018/sample_submission.csv
/kaggle/input/flight-delays-spring-2018/flight_delays_test.csv
/kaggle/input/flight-delays-spring-2018/flight_delays_train.csv


In [167]:
test = pd.read_csv('/kaggle/input/flight-delays-spring-2018/flight_delays_test.csv')
train = pd.read_csv('/kaggle/input/flight-delays-spring-2018/flight_delays_train.csv')

In [168]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


In [169]:
train.describe()

,DepTime,Distance
count,100000.000000,100000.00000
mean,1341.523880,729.39716
std,476.378445,574.61686
min,1.000000,30.00000
25%,931.000000,317.00000
50%,1330.000000,575.00000
75%,1733.000000,957.00000
max,2534.000000,4962.00000


In [170]:
train.head(10)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
5,c-8,c-3,c-4,1918,NW,MEM,MCO,683,N
6,c-1,c-27,c-4,754,DL,PBI,LGA,1035,N
7,c-4,c-29,c-6,635,OH,MSP,CVG,596,N
8,c-7,c-28,c-5,735,AA,ONT,DFW,1189,N
9,c-6,c-20,c-2,2029,OO,DEN,PSC,853,N


In [171]:
train['dep_delayed_15min'].replace({'N': 0, 'Y': 1}, inplace=True)
train.rename(columns=({'dep_delayed_15min': 'y'}), inplace=True)

X_train = train.drop('y', axis=1)
y_train = train['y']

In [172]:
train['y'].value_counts(normalize=True)

y
0    0.80956
1    0.19044
Name: proportion, dtype: float64

In [173]:
arr = set(list(train['Origin']) + list(train['Dest']) + list(test['Origin']) + list(test['Dest']))
places = pd.Series(data=list(arr)).to_dict()
places_map = {v: k for k, v in places.items()}
# places_map

In [174]:
arr = set(list(train['UniqueCarrier']) + list(test['UniqueCarrier']))
carriers = pd.Series(data=list(arr)).to_dict()
carriers_map = {v: k for k, v in carriers.items()}
# carriers_map

In [175]:
def prepare_df(df):
    df_copy = df.copy()
    df_copy['Dest'] = df_copy['Dest'].map(places_map)
    df_copy['Origin'] = df_copy['Origin'].map(places_map)
    df_copy['UniqueCarrier'] = df_copy['UniqueCarrier'].map(carriers_map)
    df_copy['Month'] = df_copy['Month'].str.replace('c-', '').astype(int)
    df_copy['DayofMonth'] = df_copy['DayofMonth'].str.replace('c-', '').astype(int)
    df_copy['DayOfWeek'] = df_copy['DayOfWeek'].str.replace('c-', '').astype(int)
    return df_copy

In [176]:
X_train = prepare_df(X_train)
test = prepare_df(test)

In [177]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train[['DepTime', 'Distance']] = scaler.fit_transform(X_train[['DepTime', 'Distance']])
test[['DepTime', 'Distance']] = scaler.fit_transform(test[['DepTime', 'Distance']])

In [179]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=314, shuffle=True)

In [182]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [190]:
n_estimators = [10, 20, 30]
max_depth = [5, 8, 12]
learning_rate = [0.1, 0.2, 0.3]

params = {'n_estimators': n_estimators,
          'max_depth': max_depth,
          'learning_rate': learning_rate}

In [191]:
grid = GridSearchCV(XGBClassifier(seed=42, nthread=4, n_jobs=-1), params, n_jobs=-1, verbose=True)

In [192]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1, nthread=4,
                                     num_parallel_tree=None, predictor=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'max_depth': [5, 8, 12],
                         'n_estimators': [10, 20, 30]},
             verbose=True)

In [193]:
grid.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=30, n_jobs=-1, nthread=4, num_parallel_tree=None,
              predictor=None, ...)

In [198]:
model_valid_pred = grid.predict_proba(X_train)[:, 1]
model_pred = grid.predict_proba(X_test)[:, 1]

In [200]:
from sklearn.metrics import roc_auc_score

In [201]:
roc_auc_score(y_test, model_pred), roc_auc_score(y_train, model_valid_pred)

(0.7382196712034219, 0.8322092593150563)

In [202]:
model_test_pred = grid.predict_proba(test)[:, 1]
pd.Series(model_test_pred, name='dep_delayed_15min').to_csv('xgb_2feat.csv', index_label='id', header=True)

In [206]:
from sklearn.metrics import f1_score


round(f1_score(y_test, grid.predict(X_test)), 3), round(f1_score(y_train, grid.predict(X_train)), 3) 

(0.19, 0.253)